In [73]:
import pandas as pd
import sqlite3 as sql

In [74]:
df = pd.read_csv('DataLake/Analista de datos M54 - kc_house_data.csv')
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


Se crearan las querys con la libreria sqlite3

In [75]:
# Crear base de datos en memoria
conexion = sql.connect(":memory:")

# Pasar df a SQL
df.to_sql("kc", conexion, index=False, if_exists="replace")


21613

In [76]:
# Realizar consulta SQL
query ="""
SELECT avg(bedrooms) as AvgHabitaciones, max(bedrooms) as MaxHabitaciones, min(bedrooms) as MinHabitaciones
FROM kc
"""

In [77]:
# imprimir resultado
resultado = pd.read_sql_query(query, conexion)
print(resultado)

   AvgHabitaciones  MaxHabitaciones  MinHabitaciones
0         3.370842               33                0


## Análisis adicionales, que respondan a preguntas que el negocio quisiera hacer.  

In [78]:
# 1. Precio promedio de una casa
query ="""
SELECT round(avg(price),2) AS PrecioPromedio
FROM kc;
"""

resultado = pd.read_sql_query(query, conexion)
print(resultado)

   PrecioPromedio
0       540088.14


In [79]:
# 2. Superficie promedio (sqft_living)
query ="""
SELECT round(avg(sqft_living),2) AS SuperficiePromedio
FROM kc;
"""
resultado = pd.read_sql_query(query, conexion)
print(resultado)

   SuperficiePromedio
0              2079.9


In [80]:
# 3. Promedio de recámaras y baños
query ="""
SELECT round(avg(bedrooms),2) AS PromedioRecamaras, round(avg(bathrooms),2) AS PromedioBaños
FROM kc;
"""
resultado = pd.read_sql_query(query, conexion)
print(resultado)

   PromedioRecamaras  PromedioBaños
0               3.37           2.11


## KPIs y datos que permitan a una persona sin conocer el negocio a fondo, darse cuenta de sus magnitudes

In [81]:
# ¿Qué meses del año tienen los precios más altos? (Abril)
query ="""
SELECT
    substr(date, 5, 2) AS month,
    avg(price) AS avg_price
FROM kc
GROUP BY month
ORDER BY avg_price DESC;
"""
resultado = pd.read_sql_query(query, conexion)
print(resultado)

   month      avg_price
0     04  561837.774989
1     06  558002.199541
2     05  550768.785833
3     07  544788.764360
4     03  543977.187200
5     10  539026.971778
6     08  536445.276804
7     09  529253.821871
8     01  525870.889571
9     12  524461.866757
10    11  521961.009213
11    02  507851.371200


In [ ]:
# ¿Las renovaciones (yr_renovated) impactan el precio? (La respuesta es que sí, las casas renovadas tienen un precio promedio más alto)
query ="""
SELECT 
    CASE WHEN yr_renovated > 0 THEN 'Renovada' ELSE 'No renovada' END AS status,
    avg(price) AS avg_price
FROM kc
GROUP BY status;
"""
resultado = pd.read_sql_query(query, conexion)
print(resultado)

        status      avg_price
0  No renovada  530360.818155
1     Renovada  760379.029540


In [98]:
# ¿Qué tipo de casas ofrecen el mejor retorno entre precio y metros cuadrados?
query ="""
SELECT round(avg(price / sqft_living), 2) || ' dlls/ft²' AS PrecioPromedioPorMetroCuadrado
FROM kc;
"""
resultado = pd.read_sql_query(query, conexion)
print(resultado)

  PrecioPromedioPorMetroCuadrado
0                264.16 dlls/ft²
